In [1]:
import pandas as pd
import numpy as np

from datetime import datetime

%matplotlib inline
pd.set_option('display.max_rows', 500)

In [2]:
pd_JH_data=pd.read_csv('COVID_relational_confirmed.csv',sep=';',parse_dates=[0])
pd_JH_data=pd_JH_data.sort_values('date',ascending=True).reset_index(drop=True).copy()
pd_JH_data.head()

,date,state,country,confirmed
0,2020-01-22,NaN,Afghanistan,0.0
1,2020-01-22,NaN,US,1.0
2,2020-01-22,NaN,Uganda,0.0
3,2020-01-22,NaN,Ukraine,0.0
4,2020-01-22,NaN,United Arab Emirates,0.0


In [3]:
test_data=pd_JH_data[((pd_JH_data['country']=='US')|
                      (pd_JH_data['country']=='Germany'))&
                     (pd_JH_data['date']>'2020-03-20')]

In [4]:
test_data.head()

,date,state,country,confirmed
16825,2020-03-21,NaN,US,26025.0
16980,2020-03-21,NaN,Germany,16662.0
17112,2020-03-22,NaN,US,34944.0
17265,2020-03-22,NaN,Germany,18610.0
17396,2020-03-23,NaN,US,46096.0


In [5]:
test_data.groupby(['country']).agg(np.max)

,date,state,confirmed
country,,,
Germany,2022-07-27,NaN,30702511.0
US,2022-07-27,NaN,90973418.0


In [6]:
# %load ../src/features/build_features.py

import numpy as np
from sklearn import linear_model
reg = linear_model.LinearRegression(fit_intercept=True)

def get_doubling_time_via_regression(in_array):
    ''' Use a linear regression to approximate the doubling rate'''

    y = np.array(in_array)
    X = np.arange(-1,2).reshape(-1, 1)

    assert len(in_array)==3
    reg.fit(X,y)
    intercept=reg.intercept_
    slope=reg.coef_

    return intercept/slope


In [7]:
test_data.groupby(['state','country']).agg(np.max)

,,date,confirmed
state,country,,


In [8]:
# this command will only work when adapting the get_doubling_time_via_regression function

#test_data.groupby(['state','country']).apply(get_doubling_time_via_regression)

In [9]:
def rolling_reg(df_input,col='confirmed'):
    ''' input has to be a data frame'''
    ''' return is single series (mandatory for group by apply)'''
    days_back=3
    result=df_input[col].rolling(
                window=days_back,
                min_periods=days_back).apply(get_doubling_time_via_regression,raw=False)
    return result

In [10]:
test_data[['state','country','confirmed']].groupby(['state','country']).apply(rolling_reg,'confirmed')

,,state,country,confirmed
state,country,,,


In [11]:
pd_DR_result=pd_JH_data[['state','country','confirmed']].groupby(['state','country']).apply(rolling_reg,'confirmed').reset_index()

In [12]:
pd_DR_result=pd_DR_result.rename(columns={'confirmed':'confirmed_DR',
                             'level_2':'index'})
pd_DR_result.head()

,state,country,index,confirmed_DR
0,Alberta,Canada,16,NaN
1,Alberta,Canada,315,NaN
2,Alberta,Canada,593,NaN
3,Alberta,Canada,872,NaN
4,Alberta,Canada,1157,NaN


In [13]:
pd_JH_data=pd_JH_data.reset_index()
pd_JH_data.head()

,index,date,state,country,confirmed
0,0,2020-01-22,NaN,Afghanistan,0.0
1,1,2020-01-22,NaN,US,1.0
2,2,2020-01-22,NaN,Uganda,0.0
3,3,2020-01-22,NaN,Ukraine,0.0
4,4,2020-01-22,NaN,United Arab Emirates,0.0


In [14]:
pd_result_larg=pd.merge(pd_JH_data,pd_DR_result[['index','confirmed_DR']],on=['index'],how='left')
pd_result_larg.head()

,index,date,state,country,confirmed,confirmed_DR
0,0,2020-01-22,NaN,Afghanistan,0.0,NaN
1,1,2020-01-22,NaN,US,1.0,NaN
2,2,2020-01-22,NaN,Uganda,0.0,NaN
3,3,2020-01-22,NaN,Ukraine,0.0,NaN
4,4,2020-01-22,NaN,United Arab Emirates,0.0,NaN


In [15]:
#pd_result_larg[pd_result_larg['country']=='Germany']

In [16]:
from scipy import signal

def savgol_filter(df_input,column='confirmed',window=5):
    ''' Savgol Filter which can be used in groupby apply function 
        it ensures that the data structure is kept'''
    window=5, 
    degree=1
    df_result=df_input
    
    filter_in=df_input[column].fillna(0) # attention with the neutral element here
    
    result=signal.savgol_filter(np.array(filter_in),
                           5, # window size used for filtering
                           1)
    df_result[column+'_filtered']=result
    return df_result
        

In [17]:
pd_filtered_result=pd_JH_data[['state','country','confirmed']].groupby(['state','country']).apply(savgol_filter).reset_index()

In [18]:
pd_result_larg=pd.merge(pd_result_larg,pd_filtered_result[['index','confirmed_filtered']],on=['index'],how='left')
pd_result_larg.head()

,index,date,state,country,confirmed,confirmed_DR,confirmed_filtered
0,0,2020-01-22,NaN,Afghanistan,0.0,NaN,NaN
1,1,2020-01-22,NaN,US,1.0,NaN,NaN
2,2,2020-01-22,NaN,Uganda,0.0,NaN,NaN
3,3,2020-01-22,NaN,Ukraine,0.0,NaN,NaN
4,4,2020-01-22,NaN,United Arab Emirates,0.0,NaN,NaN


In [19]:

pd_filtered_doubling=pd_result_larg[['state','country','confirmed_filtered']].groupby(['state','country']).apply(rolling_reg,'confirmed_filtered').reset_index()

pd_filtered_doubling=pd_filtered_doubling.rename(columns={'confirmed_filtered':'confirmed_filtered_DR',
                             'level_2':'index'})

pd_filtered_doubling.tail()

,state,country,index,confirmed_filtered_DR
81697,Zhejiang,China,260296,3547.555556
81698,Zhejiang,China,260538,2903.393939
81699,Zhejiang,China,260866,2904.424242
81700,Zhejiang,China,261108,2779.101449
81701,Zhejiang,China,261629,2906.454545


In [20]:
pd_result_larg=pd.merge(pd_result_larg,pd_filtered_doubling[['index','confirmed_filtered_DR']],on=['index'],how='left')
pd_result_larg.tail()

,index,date,state,country,confirmed,confirmed_DR,confirmed_filtered,confirmed_filtered_DR
261625,261625,2022-07-27,NaN,Kosovo,248897.0,NaN,NaN,NaN
261626,261626,2022-07-27,NaN,"Korea, South",19535242.0,NaN,NaN,NaN
261627,261627,2022-07-27,NaN,"Korea, North",1.0,NaN,NaN,NaN
261628,261628,2022-07-27,NaN,Latvia,858728.0,NaN,NaN,NaN
261629,261629,2022-07-27,Zhejiang,China,3198.0,6394.666667,3198.2,2906.454545


In [21]:
mask=pd_result_larg['confirmed']>100
pd_result_larg['confirmed_filtered_DR']=pd_result_larg['confirmed_filtered_DR'].where(mask, other=np.NaN) 

In [22]:
pd_result_larg[pd_result_larg['country']=='Germany'].tail()

,index,date,state,country,confirmed,confirmed_DR,confirmed_filtered,confirmed_filtered_DR
260371,260371,2022-07-23,NaN,Germany,30331133.0,NaN,NaN,NaN
260655,260655,2022-07-24,NaN,Germany,30331133.0,NaN,NaN,NaN
260941,260941,2022-07-25,NaN,Germany,30476605.0,NaN,NaN,NaN
261225,261225,2022-07-26,NaN,Germany,30598385.0,NaN,NaN,NaN
261508,261508,2022-07-27,NaN,Germany,30702511.0,NaN,NaN,NaN


In [23]:
pd_result_larg.to_csv('COVID_final_set.csv',sep=';',index=False)